In [208]:
task = "CCG"
prop = "1"
prop_to_prob ={"1": "100%", "0.5": "50%", "0.1": "10%", "0.25": "25%"}
model_to_compare = "BERT_Encoder"
lang = "en"
run = "Run3"

In [209]:
TI_file = f"predictions/{task}/predictions_TagInsert_{lang}_{prop}_{run}.csv"
m2c_file = f"predictions/{task}/predictions_{model_to_compare}_{lang}_{prop}_{run}.csv"

In [3]:
import os
import numpy as np
# go to parent directory
os.chdir("..")


In [210]:
# open the files
TI_preds = []
TI_accs = []
M2C_preds = []
M2C_accs = []

with open(TI_file) as f:
    # get number of lines in the file
    rows = sum(1 for line in f)
with open(TI_file) as f:
    
    for i, line in enumerate(f):
        sentence = []
        preds = line.strip().split(", ")
        # remove the last comma
        # preds[-1] = preds[-1][:-1]
        for j, pred in enumerate(preds):
            if j == len(preds) - 1 and not pred.startswith("Sentence"):
                # remove the last comma
                pred = pred[:-1]
            if pred.startswith("Sentence"):
                TI_accs.append(pred.split(": ")[1])
            elif i != rows - 1:
                word, gold, prediction, order = pred.split("|")
                sentence.append((word, gold, prediction, order))
        if len(sentence) != 0:
            TI_preds.append(sentence)

with open(m2c_file) as f:
    for i, line in enumerate(f):
        sentence = []
        preds = line.strip().split(", ")
        # remove the last comma
        # preds[-1] = preds[-1][:-1]
        for j, pred in enumerate(preds):
            if j == len(preds) - 1 and not pred.startswith("Sentence") and model_to_compare != "BERT_Encoder":
                # remove the last comma
                pred = pred[:-1]
            if pred.startswith("Sentence"):
                M2C_accs.append(pred.split(": ")[1])
            elif i != rows - 1:
                word, gold, prediction = pred.split("|")
                sentence.append((word, gold, prediction))
        if len(sentence) != 0:
            M2C_preds.append(sentence)

TI_accs = [float(acc) for acc in TI_accs]
M2C_accs = [float(acc) for acc in M2C_accs]

In [199]:
correct_ti = 0
correct_m2c = 0
total = 0
for ti_preds, m2c_preds, ti_accs, m2c_accs in zip(TI_preds, M2C_preds, TI_accs, M2C_accs):
    sentence_len = len(ti_preds)
    total += sentence_len
    for i, (ti, m2c) in enumerate(zip(ti_preds, m2c_preds)):
        ti_word, gold, ti_pred, ti_order = ti
        m2c_word, _, m2c_pred = m2c
        if ti_pred == gold:
            correct_ti += 1
        if m2c_pred == gold:
            correct_m2c += 1

print(correct_ti, correct_m2c, total, correct_ti / total, correct_m2c / total)


43046 43538 45727 0.9413694316268287 0.952128939138802


In [ ]:
import json, torch

if task == "PMB":
    tgt_to_idx = json.load(open(f"data/{task}/{lang}/processed/{task}_to_idx.json"))
    idx_to_tgt = json.load(open(f"data/{task}/{lang}/processed/idx_to_{task}.json"))
    with open(f"data/{task}/{lang}/processed/word_to_idx.json", 'r', encoding='utf-8') as f:
        word_to_idx = json.load(f)
    with open(f"data/{task}/{lang}/processed/idx_to_word.json", 'r', encoding='utf-8') as f:
        idx_to_word = json.load(f)
    train_path = f"data/{task}/{lang}/processed/train_data.pth"
    val_path = f"data/{task}/{lang}/processed/val_data.pth"
    test_path = f"data/{task}/{lang}/processed/test_data.pth"
else:
    tgt_to_idx = json.load(open(f"data/{task}/processed/{prop_to_prob[prop]}/{task}_to_idx.json"))
    idx_to_tgt = json.load(open(f"data/{task}/processed/{prop_to_prob[prop]}/idx_to_{task}.json"))
    word_to_idx = json.load(open(f"data/{task}/processed/{prop_to_prob[prop]}/word_to_idx.json"))
    idx_to_word = json.load(open(f"data/{task}/processed/{prop_to_prob[prop]}/idx_to_word.json"))
    train_path = f"data/{task}/processed/{prop_to_prob[prop]}/train_data.pth"
    val_path = f"data/{task}/processed/{prop_to_prob[prop]}/val_data.pth"
    test_path = f"data/{task}/processed/{prop_to_prob[prop]}/test_data.pth"
train_data = torch.load(train_path)
val_data = torch.load(val_path)
test_data = torch.load(test_path)

In [187]:
from collections import Counter

word_freqs = Counter()
tag_freqs = Counter()

for sentence in train_data['words']:
    for word in sentence:
        if word != word_to_idx['<PAD>']:
            word_string = idx_to_word[str(word)]
            word_freqs[word_string] += 1

for sentence in train_data['tags']:
    for tag in sentence:
        if tag != tgt_to_idx['<PAD>'] and tag != tgt_to_idx['<START>']:
            tag_string = idx_to_tgt[str(tag)]
            tag_freqs[tag_string] += 1

word_freqs.most_common(10), tag_freqs.most_common(10)

([(',', 52749),
  ('the', 44759),
  ('.', 42360),
  ('of', 24837),
  ('to', 24046),
  ('a', 20848),
  ('and', 17264),
  ('in', 16437),
  ("'s", 10145),
  ('that', 8650)],
 [('N', 182074),
  ('N/N', 151657),
  ('NP/N', 83736),
  ('PP/NP', 56438),
  (',', 52749),
  ('.', 42849),
  ('N/PP', 37774),
  ('conj', 21975),
  ('(N/N)/(N/N)', 20117),
  ('NP', 19969)])

In [211]:
normalized_orders = []
normalized_indeces = []
ti_unseen_words_accs = []
m2c_unseen_words_accs = []
for ti_preds, m2c_preds, ti_accs, m2c_accs in zip(TI_preds, M2C_preds, TI_accs, M2C_accs):
    # calculate accuracy of the two models on unseen words
    for i, (ti, m2c) in enumerate(zip(ti_preds, m2c_preds)):
        ti_word, gold, ti_pred, ti_order = ti
        m2c_word, _, m2c_pred = m2c
        if ti_word not in word_freqs:
            ti_unseen_words_accs.append(ti_pred == gold)
            normalized_order = float(ti_order) / len(ti_preds)
            normalized_orders.append(normalized_order)
            m2c_unseen_words_accs.append(m2c_pred == gold)
            normalized_index = float(i) / len(ti_preds)
            normalized_indeces.append(normalized_index)

print(f"Number of unseen words: {len(ti_unseen_words_accs)}")
print(f"TI accuracy on unseen words: {sum(ti_unseen_words_accs) / len(ti_unseen_words_accs)}")
print(f"{model_to_compare} accuracy on unseen words: {sum(m2c_unseen_words_accs) / len(m2c_unseen_words_accs)}")
print(f"Average normalized order of unseen words: {sum(normalized_orders) / len(normalized_orders)}")
print(f"Average normalized index of unseen words: {sum(normalized_indeces) / len(normalized_indeces)}")


Number of unseen words: 1618
TI accuracy on unseen words: 0.9295426452410384
BERT_Encoder accuracy on unseen words: 0.946229913473424
Average normalized order of unseen words: 0.6118882563579716
Average normalized index of unseen words: 0.45067907892956965


In [201]:
frequent_tags = [tag for tag, freq in tag_freqs.items() if freq >= 100]
common_tags = [tag for tag, freq in tag_freqs.items() if freq >= 10 and freq < 100]
rare_tags = [tag for tag, freq in tag_freqs.items() if freq < 10]

len(frequent_tags), len(common_tags), len(rare_tags)

(203, 321, 1089)

In [212]:
normalized_freq_orders, normalized_freq_indeces = [], []
normalized_common_orders, normalized_common_indeces = [], []
normalized_rare_orders, normalized_rare_indeces = [], []
normalized_oov_orders, normalized_oov_indeces = [], []
ti_freq_tags_accs, m2c_freq_tags_accs = [], []
ti_common_tags_accs, m2c_common_tags_accs = [], []
ti_rare_tags_accs, m2c_rare_tags_accs = [], []
ti_oov_tags_accs, m2c_oov_tags_accs = [], []
for ti_preds, m2c_preds, ti_accs, m2c_accs in zip(TI_preds, M2C_preds, TI_accs, M2C_accs):
    for i, (ti, m2c) in enumerate(zip(ti_preds, m2c_preds)):
        ti_word, gold, ti_pred, ti_order = ti
        m2c_word, _, m2c_pred = m2c
        if gold not in frequent_tags and gold not in common_tags and gold not in rare_tags:
            ti_oov_tags_accs.append(ti_pred == gold)
            normalized_order = float(ti_order) / len(ti_preds)
            normalized_oov_orders.append(normalized_order)
            m2c_oov_tags_accs.append(m2c_pred == gold)
            normalized_index = float(i) / len(ti_preds)
            normalized_oov_indeces.append(normalized_index)
        elif gold in frequent_tags:
            ti_freq_tags_accs.append(ti_pred == gold)
            normalized_order = float(ti_order) / len(ti_preds)
            normalized_freq_orders.append(normalized_order)
            m2c_freq_tags_accs.append(m2c_pred == gold)
            normalized_index = float(i) / len(ti_preds)
            normalized_freq_indeces.append(normalized_index)
        elif gold in common_tags:
            ti_common_tags_accs.append(ti_pred == gold)
            normalized_order = float(ti_order) / len(ti_preds)
            normalized_common_orders.append(normalized_order)
            m2c_common_tags_accs.append(m2c_pred == gold)
            normalized_index = float(i) / len(ti_preds)
            normalized_common_indeces.append(normalized_index)
        elif gold in rare_tags:
            ti_rare_tags_accs.append(ti_pred == gold)
            normalized_order = float(ti_order) / len(ti_preds)
            normalized_rare_orders.append(normalized_order)
            m2c_rare_tags_accs.append(m2c_pred == gold)
            normalized_index = float(i) / len(ti_preds)
            normalized_rare_indeces.append(normalized_index)

print(f"Number of frequent tags: {len(ti_freq_tags_accs)}")
print(f"TI accuracy on frequent tags: {sum(ti_freq_tags_accs) / len(ti_freq_tags_accs)}")
print(f"{model_to_compare} accuracy on frequent tags: {sum(m2c_freq_tags_accs) / len(m2c_freq_tags_accs)}")
print(f"Average normalized order of frequent tags: {sum(normalized_freq_orders) / len(normalized_freq_orders)}")
print(f"Average normalized index of frequent tags: {sum(normalized_freq_indeces) / len(normalized_freq_indeces)}")
print(f"Number of common tags: {len(ti_common_tags_accs)}")
print(f"TI accuracy on common tags: {sum(ti_common_tags_accs) / len(ti_common_tags_accs)}")
print(f"{model_to_compare} accuracy on common tags: {sum(m2c_common_tags_accs) / len(m2c_common_tags_accs)}")
print(f"Average normalized order of common tags: {sum(normalized_common_orders) / len(normalized_common_orders)}")
print(f"Average normalized index of common tags: {sum(normalized_common_indeces) / len(normalized_common_indeces)}")
print(f"Number of rare tags: {len(ti_rare_tags_accs)}")
if len(ti_rare_tags_accs) > 0:
    print(f"TI accuracy on rare tags: {sum(ti_rare_tags_accs) / len(ti_rare_tags_accs)}")
    print(f"{model_to_compare} accuracy on rare tags: {sum(m2c_rare_tags_accs) / len(m2c_rare_tags_accs)}")
    print(f"Average normalized order of rare tags: {sum(normalized_rare_orders) / len(normalized_rare_orders)}")
    print(f"Average normalized index of rare tags: {sum(normalized_rare_indeces) / len(normalized_rare_indeces)}")
if len(ti_oov_tags_accs) > 0:
    print(f"Number of OOV tags: {len(ti_oov_tags_accs)}")
    print(f"TI accuracy on OOV tags: {sum(ti_oov_tags_accs) / len(ti_oov_tags_accs)}")
    print(f"{model_to_compare} accuracy on OOV tags: {sum(m2c_oov_tags_accs) / len(m2c_oov_tags_accs)}")
    print(f"Average normalized order of OOV tags: {sum(normalized_oov_orders) / len(normalized_oov_orders)}")
    print(f"Average normalized index of OOV tags: {sum(normalized_oov_indeces) / len(normalized_oov_indeces)}")


Number of frequent tags: 45191
TI accuracy on frequent tags: 0.9474674160784227
BERT_Encoder accuracy on frequent tags: 0.9573366378261158
Average normalized order of frequent tags: 0.516362990237016
Average normalized index of frequent tags: 0.48004863567831235
Number of common tags: 428
TI accuracy on common tags: 0.5093457943925234
BERT_Encoder accuracy on common tags: 0.6401869158878505
Average normalized order of common tags: 0.9040693338358383
Average normalized index of common tags: 0.40202082762790475
Number of rare tags: 92
TI accuracy on rare tags: 0.010869565217391304
BERT_Encoder accuracy on rare tags: 0.07608695652173914
Average normalized order of rare tags: 0.8981887149276789
Average normalized index of rare tags: 0.3787890212260492
Number of OOV tags: 16
TI accuracy on OOV tags: 0.0
BERT_Encoder accuracy on OOV tags: 0.0
Average normalized order of OOV tags: 0.8728169714950174
Average normalized index of OOV tags: 0.48178755524046696


In [110]:
normalized_orders = []
normalized_indeces = []
right_for_ti = []
wrong_for_m2c = []
sentences_right_for_ti = []
for ti_preds, m2c_preds, ti_accs, m2c_accs in zip(TI_preds, M2C_preds, TI_accs, M2C_accs):
    sentence_len = len(ti_preds)
    for i, (ti, m2c) in enumerate(zip(ti_preds, m2c_preds)):
        ti_word, gold, ti_pred, ti_order = ti
        m2c_word, _, m2c_pred = m2c
        if ti_pred == gold and m2c_pred != gold:
            if (ti_preds, m2c_preds) not in sentences_right_for_ti:
                sentences_right_for_ti.append((ti_preds, m2c_preds))
            right_for_ti.append(ti_pred)
            wrong_for_m2c.append(m2c_pred)
            normalized_index = float(i) / sentence_len
            normalized_indeces.append(normalized_index)
            normalized_order = float(ti_order) / sentence_len
            normalized_orders.append(normalized_order)

right_counter = Counter(right_for_ti)
wrong_counter = Counter(wrong_for_m2c)

print(f"Number of tags where TI is right and {model_to_compare} is wrong: {len(right_for_ti)}")
print(f"Number of unique tags where TI is right and {model_to_compare} is wrong: {len(right_counter)}")
print(f"Average normalized order of words where TI is right and {model_to_compare} is wrong: {sum(normalized_orders) / len(normalized_orders)}")
print(f"Average normalized index of words where TI is right and {model_to_compare} is wrong: {sum(normalized_indeces) / len(normalized_indeces)}")
print(f"Most common TI preds where TI is right and {model_to_compare} is wrong: {right_counter.most_common(10)}")
print(f"Most common {model_to_compare} preds where TI is right and {model_to_compare} is wrong: {wrong_counter.most_common(10)}")

Number of tags where TI is right and BERT_Encoder is wrong: 27
Number of unique tags where TI is right and BERT_Encoder is wrong: 24
Average normalized order of words where TI is right and BERT_Encoder is wrong: 0.912066431510876
Average normalized index of words where TI is right and BERT_Encoder is wrong: 0.3610082304526749
Most common TI preds where TI is right and BERT_Encoder is wrong: [('(NP\\NP)/NP', 3), ('(S[ng]\\NP)/PP', 2), ('(S[dcl]\\PP)/NP', 1), ('(S[adj]\\NP)/(S[adj]\\NP)', 1), ('S[q]\\S[dcl]', 1), ('(S[dcl]\\NP)/(S[b]\\NP)', 1), ('(S[b]\\NP)/(S[adj]\\NP)', 1), ('(S[pt]\\NP)\\NP', 1), ('(S[b]\\NP)/NP', 1), ('(S[dcl]\\NP)/PP', 1)]
Most common BERT_Encoder preds where TI is right and BERT_Encoder is wrong: [('(S[dcl]\\NP)/NP', 5), ('(S[dcl]\\NP)/(S[dcl]\\NP)', 3), ('S[pt]\\NP', 2), ('((S[dcl]\\NP)\\(S[dcl]\\NP))/NP', 2), ('S[ng]\\NP', 2), ('(S[dcl]\\NP)/(S[ng]\\NP)', 2), ('N\\N', 2), ('PP/NP', 2), ('S[adj]\\NP', 1), ('(S[dcl]\\NP)/(S[adj]\\NP)', 1)]


In [112]:
# count how many times both model get wrong a comma or a period
normalized_orders = []
normalized_indeces = []
ti_wrong_comma_period = []
m2c_wrong_comma_period = []
for ti_preds, m2c_preds, ti_accs, m2c_accs in zip(TI_preds, M2C_preds, TI_accs, M2C_accs):
    sentence_len = len(ti_preds)
    for i, (ti, m2c) in enumerate(zip(ti_preds, m2c_preds)):
        ti_word, gold, ti_pred, ti_order = ti
        m2c_word, _, m2c_pred = m2c
        if gold in [',', '.']:
            if ti_pred != gold:
                ti_wrong_comma_period.append(gold)
                normalized_order = float(ti_order) / sentence_len
                normalized_orders.append(normalized_order)
            if m2c_pred != gold:
                m2c_wrong_comma_period.append(gold)
                normalized_index = float(i) / sentence_len
                normalized_indeces.append(normalized_index)

ti_wrong_comma_period_counter = Counter(ti_wrong_comma_period)
m2c_wrong_comma_period_counter = Counter(m2c_wrong_comma_period)

print(f"Number of commas and periods where TI is wrong: {len(ti_wrong_comma_period)}")
print(f"Number of commas and number of periods where TI is wrong: {ti_wrong_comma_period_counter}")
if len(normalized_orders) > 0:
    print(f"Average normalized order of commas and periods where TI is wrong: {sum(normalized_orders) / len(normalized_orders)}")
print(f"Number of commas and periods where {model_to_compare} is wrong: {len(m2c_wrong_comma_period)}")
print(f"Number of commas and number of periods where {model_to_compare} is wrong: {m2c_wrong_comma_period_counter}")
if len(normalized_indeces) > 0: 
    print(f"Average normalized index of commas and periods where {model_to_compare} is wrong: {sum(normalized_indeces) / len(normalized_indeces)}")



Number of commas and periods where TI is wrong: 0
Number of commas and number of periods where TI is wrong: Counter()
Number of commas and periods where BERT_Encoder is wrong: 0
Number of commas and number of periods where BERT_Encoder is wrong: Counter()


In [113]:
len(sentences_right_for_ti)
with open(f"predictions/TI_right_{model_to_compare}_wrong_{task}_{prop}.txt", "w") as f:
    for i, (ti_preds, m2c_preds) in enumerate(sentences_right_for_ti):
        f.write(f"{i}): ")
        for ti, m2c in zip(ti_preds, m2c_preds):
            ti_word, gold, ti_pred, ti_order = ti
            m2c_word, _, m2c_pred = m2c
            if ti_pred == gold and m2c_pred != gold:
                f.write(f"{ti_word.upper()}|{gold.upper()}|{ti_pred.upper()}|{ti_order.upper()}|{m2c_pred.upper()} ")
            else:
                f.write(f"{ti_word}|{gold}|{ti_order}| ")
        f.write("\n")